In [1]:
from dotenv import load_dotenv

In [2]:
# Load Environment Variables
load_dotenv()

True

In [3]:
# Create an engine to connect to the database

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)
db_type="sqlite"

aengine = create_async_engine(
    url=f"{db_type}+aiosqlite:///demo_databases/tn_covid_cases_11_may.sqlite"
)

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [8]:
# Parameters
metric_db_id = "test"
llm = "gpt-4o"
validation_llm = "gpt-4o"
guardrails_llm = "gpt-4o"
sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."
db_description = "- bed_vacancies_clinics_11_may: Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"
num_common_values = 10
indicator_vars="district_name" # This should be a comma delimited string in multiple vars


### Single-turn question

In [12]:
# Your question
query = {
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
}


In [13]:
from askametric.query_processor.query_processor import LLMQueryProcessor

async with async_session() as session:
    qp = LLMQueryProcessor(
        query,
        session,
        metric_db_id,
        db_type,
        llm,
        guardrails_llm,
        sys_message,
        db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()
    print(qp.final_answer)

In Chennai, there are a total of 20,334 beds available. This total is calculated by adding 7,179 beds from clinics and 13,155 beds from health centers and district hospitals. This information is based on the data collected as of May 11.


### Multi-turn chat

In [ ]:
# Simulating a conversation
queries = [{
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
},
{
    "query_text": "How about Ranipet??",
    "query_metadata": {}
},
{
    "query_text": "And how many COVID patients there??",
    "query_metadata": {}
},
{
    "query_text": "How many beds with oxygen supply in Ariyalur??",
    "query_metadata": {}
},
{
    "query_text": "And how many COVID patients?",
    "query_metadata": {}
},
{
    "query_text": "No I meant in Ranipet",
    "query_metadata": {}
},]

In [9]:
from askametric.query_processor.query_processor import MultiTurnQueryProcessor

async with async_session() as session:
    mqp = MultiTurnQueryProcessor(
        asession=session,
        metric_db_id=metric_db_id,
        db_type=db_type,
        llm=llm,
        guardrails_llm=guardrails_llm,
        sys_message=sys_message,
        db_description=db_description,
        column_description="",
        indicator_vars=indicator_vars,
        num_common_values=num_common_values,
        chat_memory_length=5
    )
    results = []
    for query in queries:
        print(f"Q: {query['query_text']}")
        result = await mqp.process_query(query)
        results.append(result)
        print(f"A: {result.final_answer}")
        print("\n")

Q: How many beds are there in chennai??
A: In Chennai, there are a total of 20,334 beds available. This total is calculated by adding 7,179 beds from clinics and 13,155 beds from health centers and district hospitals. This information is based on the data collected as of May 11.


Q: How about Ranipet??
A: In Ranipet, there are a total of 243 vacant beds available. This number is the sum of 153 vacant beds from clinics and 90 vacant beds from health centers and district hospitals. This information is based on the latest data collected from these facilities.


Q: And how many COVID patients there??
A: As of the data available for May 11, there are 162,181 COVID patients under treatment in Tamil Nadu. This number was obtained by summing up the total number of persons under treatment on that date from the collected data.


Q: How many beds with oxygen supply in Ariyalur??
A: There are 314 beds with oxygen supply available in Ariyalur. This information was obtained from the database that t